In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
# HELPER FUNCTIONS

# INIT WEIGHTS

In [28]:
def init_weights(shape):
    init_random_dist=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [6]:
# INIT BIAS

In [30]:
def init_bias(shape):
    init_bias_vals=tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [7]:
# CONV2D

In [12]:
def conv2d(x,W):
    # x --> [batch,height(H),width(W),Channels]
    # W --> [filter H,Filter W, Channels IN, Channels Out]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME') # same means 0 padding

In [8]:
# POOLING

In [14]:
def max_pool_2by2(x):
    # x --> [batch,h,w,c]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # ksize = size of the window i.e. shape of the filter

In [34]:
# CONVOLUTIONAL LAYER
def convolutional_layer(input_x,shape):
    W=init_weights(shape)
    b=init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [2]:
# Normal (Fully Connected)
def normal_full_layer(input_layer,size):
    input_size=int(input_layer.get_shape()[1])
    W=init_weights([input_size,size])
    b= init_bias([size])
    return tf.matmul(input_layer,W)+ b 

In [17]:
# PLACEHOLDERS
x=tf.placeholder(tf.float32,shape=[None,784 ])

In [18]:
y_true=tf.placeholder(tf.float32,shape=[None,10])

In [19]:
# LAYERS
x_image= tf.reshape(x,[-1,28,28,1])

In [35]:
convo_1=convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling=max_pool_2by2(convo_1)

In [37]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [41]:
convo_2_flat= tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one=tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [42]:
# DROPOUT
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [43]:
y_pred=normal_full_layer(full_one_dropout,10)

In [44]:
# LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [48]:
# OPTIMIZER
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [49]:
init=tf.global_variables_initializer()

In [ ]:
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x , batch_y=mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if i%100 == 0:
            print("On Step: {}".format(i))
            print("Accuracy: ")
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc=tf.reduce_mean(tf.cast(matches,tf.float32))
            
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print()

On Step: 0
Accuracy: 
0.0966

On Step: 100
Accuracy: 
0.942

On Step: 200
Accuracy: 
0.9571

On Step: 300
Accuracy: 
0.9661

